# Interfacing Julia with other languages

## Calling other languages from Julia

* Native function calls to C and Fortran
* packages for other languages (Python, C++, R, Java...)

## Calling Julia from other languages

* Embed Julia in a C library
* Embedding for some languages (Python, R)
* Build binaries from Julia code

# Calling other languages from Julia

Several examples are taken from the "_Julia is fast_" (available [here](https://github.com/JuliaComputing/JuliaBoxTutorials/blob/master/introductory-tutorials/intro-to-julia/09.%20Julia%20is%20fast.ipynb))

## Calling C and Fortran code

Official documentation: https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/

C functions can be called directly from Julia with little to no overhead.
The code must be available as a shared library.

Shared libraries and functions are referenced by `(:function, "library")`:
* `function` is the C-exported function name.
* `library` refers to the shared library name

Sample C code: compute the sum of an array of `double`

In [1]:
using Libdl
const Clib = tempname()   # make a temporary file

"/tmp/juliaO08YAF"

The code below takes an array of `double` as input, and returns the sum of its elements.

_Note: the sole purpose of this sample code is to generate a shared library that will be called later on.
You do not need to understand C to continue reading_

In [2]:
# Sample C code
# This function computes the sum of an array of double
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""
;

In [3]:
# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

You can use `ccall` to actually generate a call to the library function. Arguments to `ccall` are as follows:

1. A `(:function, "library")` pair, which must be written as a literal constant
2. Return type
3. A tuple of input types. The input types must be written as a literal tuple, not a tuple-valued variable or expression.
4. The following arguments, if any, are the actual argument values passed to the function.

Calling a C function is done using `ccall`.

The `ccall` syntax can be heavy to read, so it is common to wrap the C function into a Julia function.

In [4]:
# Wrap the C function in a Julia function
c_sum(X::Array{Float64}) = ccall(
    ("c_sum", Clib),          # The C-function's name and the library it comes from
    Float64,                  # Return type
    (Csize_t, Ptr{Float64}),  # Tuple of input types
    length(X), X              # actual arguments
)

c_sum (generic function with 1 method)

In [5]:
u = ones(10)
c_sum(u)

10.0

## Calling Python from Julia

`PyCall` is the default package for calling Python from Julia: https://github.com/JuliaPy/PyCall.jl

>  PyCall provides the ability to directly call and fully interoperate with Python from the Julia language.

> You can import arbitrary Python modules from Julia, call Python functions (with automatic conversion of types between Julia and Python), define Python classes from Julia methods, and share large data structures between Julia and Python without copying them.

Built-in Python functions can be imported directly

In [6]:
# using Pkg; Pkg.add("PyCall")
using PyCall

In [7]:
py_sum = pybuiltin("sum")  # Python's built-in `sum` function
py_sum(u)

10.0

You can also import functions from a Python package, e.g. `numpy`

In [8]:
# using Pkg; Pkg.add("Conda")  # un-comment this line to install Conda
using Conda

In [9]:
# Conda.add("numpy")  # un-comment this line to install numpy

In [10]:
np_sum = pyimport("numpy")["sum"]  # Numpy's `sum` function, i.e., `numpy.sum`
np_sum(u)

10.0

## Other languages (C++, R, Java, etc)

### Calling C++ from Julia
* `Cxx` : https://github.com/Keno/Cxx.jl
* `CxxWrap` : https://github.com/JuliaInterop/CxxWrap.jl


### Calling Java from Julia

`JavaCall` https://github.com/JuliaInterop/JavaCall.jl


### Calling R from Julia

`RCall` https://github.com/JuliaInterop/RCall.jl


### Calling other languages from Julia

https://github.com/JuliaInterop, or use a C API

# Embedding Julia

## The C API

The following example is taken from https://docs.julialang.org/en/v1/manual/embedding/

```c
#include <julia.h>
JULIA_DEFINE_FAST_TLS() // only define this once, in an executable (not in a shared library) if you want fast code

int main(int argc, char *argv[])
{
    /* required: setup the Julia context */
    jl_init();

    /* run Julia commands */
    jl_eval_string("print(sqrt(2.0))");

    /* strongly recommended: notify Julia that the
         program is about to terminate. this allows
         Julia time to cleanup pending write requests
         and run all finalizers
    */
    jl_atexit_hook(0);
    return 0;
}
```

## Calling Julia from Python

Install `PyJulia`: https://github.com/JuliaPy/pyjulia

You can import and call julia functions directly in Python

```py
>>> from julia import Base  # import julia's Base

>>> Base.cos(1.0)           # call julia functions as usual
0.5403023058681398
```

## Other languages

### Using the C API

### Specific interfaces

### Building binaries
* (ongoing) `PackageCompiler` https://github.com/JuliaLang/PackageCompiler.jl